In [1]:
# Library
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import DictionaryLearning, sparse_encode
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import SparseCoder

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

# Load data
measurement = np.load('../../dataset/meas_symm_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:1000]  # Using 1000 samples for simplicity

# Split data
trainCIR, testCIR = train_test_split(data_cir, test_size=0.2, random_state=42)

# Define channels
alice_channel = 3  # Channel 3 is ALICE (legitimate)
eve_channel = 6    # Channel 6 is EVE (illegitimate)

# Set the number of dictionary atoms (components) and sparsity level
# N_components = 3  # Number of Sparse Components
n_components = 20 # Adjust this number based on your data
n_nonzero_coefs = 5  # Number of non-zero coefficients in sparse coding

In [13]:
def create_sparse_representation(data, n_components, n_nonzero_coefs):
    """
    Create a sparse representation with reduced dimensions.
    
    Parameters:
    - data: Input data with shape (n_samples, n_data_points, n_features)
    - n_components: Reduced dimensionality
    - n_nonzero_coefs: Sparsity level
    
    Returns:
    - Sparse representation with reduced dimensions.
    """
    n_samples, n_data_points, n_features = data.shape

    # Initialize sparse representation array
    sparse_representation = np.zeros((n_samples, n_components, n_features))
    print(sparse_representation.shape)
    # Process each feature separately
    for feature in range(n_features):
        # Extract data for current feature
        feature_data = data[:, :, feature]  # Shape: (n_samples, n_data_points)
        print(feature_data.shape)
        # Learn a dictionary (or use a pre-defined one)
        dict_learner = DictionaryLearning(
            n_components=n_components, 
            transform_algorithm='lasso_lars', 
            transform_n_nonzero_coefs=n_nonzero_coefs
        )
        dict_learner.fit(feature_data)
        sparse_codes = dict_learner.transform(feature_data)
        print(sparse_codes.shape)

        # Store the sparse codes for this feature
        sparse_representation[:, :, feature] = sparse_codes

    return sparse_representation

In [14]:
# ----------------------------------------------------- Preprocessing -----------------------------------------------------
# ----------------- Training data -----------------
train_alice_cirs = trainCIR[:, alice_channel, :, :]  # Alice's CIRs
train_eve_cirs = trainCIR[:, eve_channel, :, :]      # Eve's CIRs
train_data_combined  = np.vstack((train_alice_cirs, train_eve_cirs))
print('train_data_combined.shape:', train_data_combined.shape)

# Create sparse representation
train_sparse_cirs = create_sparse_representation(train_data_combined, n_components, n_nonzero_coefs)
print('train_sparse_cirs.shape:', train_sparse_cirs.shape)

train_data_combined.shape: (1600, 251, 2)
(1600, 20, 2)
(1600, 251)
(1600, 20)
(1600, 251)
(1600, 20)
train_sparse_cirs.shape: (1600, 20, 2)


In [5]:
test_alice_CIRs = testCIR[:, alice_channel, :, :]
test_eve_CIRs = testCIR[:, eve_channel, :, :]
test_cirs = np.vstack((test_alice_CIRs, test_eve_CIRs))

test_sparse_cirs = create_sparse_representation(test_cirs, n_components, n_nonzero_coefs)
print('test_cirs.shape:', test_sparse_cirs.shape)
test_sparse_cirs_reshaped = test_sparse_cirs.reshape(test_sparse_cirs.shape[0], -1)
print('Test data shape:', test_sparse_cirs_reshaped.shape)

test_cirs.shape: (400, 20, 2)
Test data shape: (400, 40)


In [6]:
# Labels for training data
train_alice_labels = np.zeros(train_alice_cirs.shape[0])  # Label '0' for Alice.
train_eve_labels = np.ones(train_eve_cirs.shape[0])       # Label '1' for Eve.
train_labels = np.hstack((train_alice_labels, train_eve_labels))

# Labels for test data
test_alice_labels = np.zeros(test_alice_CIRs.shape[0])  # Label '0' for Alice.
test_eve_labels = np.ones(test_eve_CIRs.shape[0])       # Label '1' for Eve.
test_labels = np.hstack((test_alice_labels, test_eve_labels))

In [7]:
# Dictionary
atoms = train_sparse_cirs.reshape(train_sparse_cirs.shape[0], -1)
D = atoms.T
print('Dictionary shape:', D.shape)

Dictionary shape: (40, 1600)


In [8]:
# ---------------------------------------------- Sparse Coding & Classification ------------------------------------------------
# Step 5: Sparse Coding Function
def find_sparse_coefficients(tSample, D, n_nonzero_coefs=5):
    # print(f'tSample: {tSample.shape}')
    omp = OrthogonalMatchingPursuit(n_nonzero_coefs=n_nonzero_coefs)
    omp.fit(D, tSample)
    return omp.coef_

# Step 6: Function to Calculate Residuals for Each Class
def calculate_residual(tSample, D, coefficients, class_indices):
    coef_class = np.zeros_like(coefficients)
    coef_class[class_indices] = coefficients[class_indices]  # Keep onltSample coefficients for the specified class
    reconstructed_signal = D @ coef_class
    residual = np.linalg.norm(tSample - reconstructed_signal)
    return residual

# Step 7: Classification Function
def classify_signal(tSample, D, trainLabel):
    
    # Find sparse coefficients for the new signal
    coefficients = find_sparse_coefficients(tSample, D)
    
    
    # Initialize residuals list
    residuals = []

    # Calculate residual for each class
    unique_classes = np.unique(trainLabel) # 0 and 1
    for class_label in unique_classes:
        class_indices = np.where(trainLabel == class_label)[0]  # Indices of columns in D belonging to this class
        residual = calculate_residual(tSample, D, coefficients, class_indices)
        residuals.append(residual)

    # Predict the class with the smallest residual
    predicted_class = unique_classes[np.argmin(residuals)]
    return predicted_class

# Step 8: Classifying Test Data and Evaluating the Model
predictions = []

for cir in test_sparse_cirs_reshaped:
    predicted_class = classify_signal(cir, D, train_labels)
    predictions.append(predicted_class)

predictions = np.array(predictions)
# print(predictions.shape)


In [9]:
from sklearn.metrics import accuracy_score, confusion_matrix

# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(test_labels, predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# Calculate MDR, FAR, AR
MDR = fp / (fp + tn) if (fp + tn) > 0 else 0
FAR = fn / (fn + tp) if (fn + tp) > 0 else 0
gamma = (tp + fn) / (tn + fp) if (tn + fp) > 0 else 0
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp)) if ((tp + fn) + gamma * (tn + fp)) > 0 else 0

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")


Classification Accuracy: 53.50%
tp: 170
tn: 44
fp: 156
fn: 30
MDR: 0.78
FAR: 0.15
AR: 0.535
